# This notebook converts the KML file into CSV file containing the eateries

In [1]:
import re
from pykml import parser
import pandas as pd

In [2]:
def getEateryInfo(kmlDescription,requiredFields):
    eateryInfo = {}
    delimeters = '|'.join([f'({field})'for field in requiredFields])
    kmlDescription = re.split(delimeters,str(kmlDescription))
    kmlDescription = list(filter(lambda a: a != None, kmlDescription))
    for i,obj in enumerate(kmlDescription):
        if obj in requiredFields:
            eateryInfo[obj]=kmlDescription[i+1].split('</td>')[0].split('<td>')[-1]
    return eateryInfo
def clean_coordinates_col(coord):
    coord = coord.split(',')
    return f'{coord[1]},{coord[0]}'
def clean_name_col(NAME):
    return NAME.split('-')[0]
def clean_unit_col(UNIT):
    return UNIT.split(',')[0]

In [3]:
filename=r"C:\Users\tanch\AndroidStudioProjects\cz2006_app\EateryData\healthier-eateries.kml"
with open(filename) as f:
    folder = parser.parse(f).getroot().Document.Folder
    
requiredFields = ['ADDRESSBLOCKHOUSENUMBER','ADDRESSBUILDINGNAME','ADDRESSPOSTALCODE','ADDRESSSTREETNAME','NAME','ADDRESSFLOORNUMBER','ADDRESSUNITNUMBER']

cordi=[]
db=pd.DataFrame()
for pm in folder.Placemark:
    plcs1=pm.Point.coordinates
    cordi.append(plcs1.text)
    db = db.append(getEateryInfo(pm.description,requiredFields),ignore_index=True)
db['coordinates']=cordi
db['coordinates'] = db.coordinates.apply(clean_coordinates_col)
db['NAME'] = db.NAME.apply(clean_name_col)
db['ADDRESSUNITNUMBER'] = db.ADDRESSUNITNUMBER.apply(clean_unit_col)
db.rename(columns = {'NAME':'name'},inplace=True)
db['address'] = db.ADDRESSBUILDINGNAME+' '+db.ADDRESSSTREETNAME+' #'+db.ADDRESSFLOORNUMBER+'-'+db.ADDRESSUNITNUMBER+' Singapore '+db.ADDRESSPOSTALCODE

In [4]:
db[['latitude','longitude']] = db['coordinates'].str.split(',',1,expand =True)

In [5]:
relevant_col = ['name','address','latitude','longitude']

In [7]:
db[relevant_col].to_csv(r'C:\Users\tanch\Documents\NTU Year 2\Sem 2\CZ2006, Software Engineering\Project codes\Data\HealthyEateries.csv',index = False)

In [6]:
to_json = {'eateries':list(db[relevant_col].T.to_dict().values())};to_json

{'eateries': [{'name': "McDonald's ",
   'address': 'Bugis Junction Victoria Street #1-K5 Singapore 188021',
   'latitude': '1.29911316126836',
   'longitude': '103.855411151136'},
  {'name': "McDonald's ",
   'address': 'Bugis Village Rochor Road #1-42 Singapore 188426',
   'latitude': '1.30089884539002',
   'longitude': '103.855242173644'},
  {'name': "McDonald's ",
   'address': ' Bukit Batok Central #1-10 Singapore 650632',
   'latitude': '1.34941218045919',
   'longitude': '103.750538659689'},
  {'name': "McDonald's ",
   'address': ' Bukit Batok Street 11 #1-451 Singapore 650152',
   'latitude': '1.34830189004312',
   'longitude': '103.744257269677'},
  {'name': "McDonald's ",
   'address': ' Bukit Batok Street 31 #1-27 Singapore 650374',
   'latitude': '1.35809927236864',
   'longitude': '103.750719677477'},
  {'name': "McDonald's ",
   'address': 'Bukit Panjang Plaza Jelebu Road #2-24 Singapore 677743',
   'latitude': '1.38051732815913',
   'longitude': '103.763961852618'},
  {

In [8]:
import json
with open(r'C:\Users\tanch\AndroidStudioProjects\cz2006_app\EateryData\HealthyEateries.json', "w", encoding="utf8") as json_file:
    json_file.write(json.dumps(to_json, indent = 4)) 
    